# Bangumi Takeout

项目主页：[jerrylususu/bangmui-takeout-py](https://github.com/jerrylususu/bangumi-takeout-py)

## 运行步骤

1. 请点击菜单栏「代码执行程序 - 全部运行」（繁體中文「執行階段－全部執行」）
2. 卷动到页面底部，点击认证链接并选择「允许」，将下一个页面中的 `access_token` 复制到页面底部的表单中 Token 填写处
3. 选择数据源，具体请参见项目主页，如无特殊要求保持默认即可
4. 如果有之前导出的 `takeout.json` 文件，可点击「Upload」上传，此时会采用增量方式更新收视进度，能极大提升导出速度。如无可忽略。
5. 选择导出格式：`json`, `html`，`csv`
5. 最后点击「Run」以启动导出过程

## 常见问题

表单可能需要稍等一会才会显示（需要等待运行时分配，进度会在右上角显示），请耐心等待，最长应该不会超过一分钟。

如无特殊需求，数据源建议保持默认的「Bangumi Archive」以减小对服务器的压力。关于两个数据源之间的差异，请参见本项目主页的「数据源」一节。

如果选择上传此前的 `takeout.json`，上传过程中没有进度显示，需要稍等一会。也可以用 Colab 界面左侧的文件面板上传。

执行完成后会自动触发 `takeout.json`, `takeout.html` 或 `takeout.csv` 的下载，也可以从左侧文件侧边栏（最后一个图标）中手动下载。如果浏览器提示「下载多个文件」，请选择允许。

In [1]:
from ipywidgets import interact, widgets
from IPython.display import display
import json
from google.colab import files

token_html = widgets.HTML(value='<a href="https://api.bgm.tv/v0/oauth/redirect" target="_blank">Click here to generate access_token.</a>')
display(token_html)

text = widgets.Text(value='',placeholder='Paste access_token here!',description='Token:',disabled=False)
display(text)

select = widgets.Dropdown(options=['Bangumi API', 'Bangumi Archive'],value='Bangumi Archive',description='Source:',disabled=False,)
display(select)

existing_upload = widgets.FileUpload(accept='.json',multiple=False)
display(existing_upload)

checkbox_as_json = widgets.Checkbox(value=True, description='JSON', indent=True)
display(checkbox_as_json)
checkbox_as_html = widgets.Checkbox(value=True, description='HTML', indent=True)
display(checkbox_as_html)
checkbox_as_csv = widgets.Checkbox(value=True, description='CSV', indent=True)
display(checkbox_as_csv)


button = widgets.Button(tooltip='Run',description="Run")
display(button)

def callback(wdgt):
    print("uploading... may take a while...")
    with open("takeout.json", "wb") as fp:
      for k, v in existing_upload.value.items():
        fp.write(v["content"])
        break
    print("upload finished!")

    !git clone https://github.com/jerrylususu/bangumi-takeout-py
    !pip install requests tqdm
    !mv bangumi-takeout-py/* .
    !rm -rf bangumi-takeout-py
    !touch no_gui

    import fetch
    import generate_html
    import generate_csv

    if select.value == "Bangumi Archive":
      print("downloading archive data")
      wget_result = !wget https://github.com/bangumi/Archive/releases/download/archive/dump.zip
      unzip_result = !unzip dump.zip

    with open(".bgm_token","w",encoding="u8") as f:
      tokens = {"access_token": text.value}
      json.dump(tokens, f)
    
    fetch.main()
    if checkbox_as_json.value:
      files.download('takeout.json')

    if checkbox_as_html.value:
      generate_html.main()
      files.download('takeout.html') 

    if checkbox_as_csv.value:
      generate_csv.main()
      files.download('takeout.csv')

button.on_click(callback)